In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install huggingface
!pip install transformers
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use("ggplot")
import transformers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import logging
logging.basicConfig(level= logging.INFO)
import pickle
import time
import random
import datetime
pd.set_option("display.max_columns", 500)

     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.4 MB/s 
     |████████████████████████████████| 59 kB 7.0 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
     |████████████████████████████████| 895 kB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid_vaccine_symptoms.csv")
device = torch.device("cuda")

In [ ]:
features = ["vax_manu", "vax_dose_series", "state", "age_yrs", "sex", "lab_data", "other_meds", "cur_ill", "history", "prior_vax", "allergies"]
X = df[features] 
y = df["serious_se"].apply(lambda x: 0 if x == "not serious" else 
                                1 if x == "serious" else
                                2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=3258)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 5829)

X_train_sen = X_train.fillna("").apply(lambda row: " [SEP] ".join(row.values.astype(str)), axis = 1)
X_val_sen = X_val.fillna("").apply(lambda row: " [SEP] ".join(row.values.astype(str)), axis = 1)
X_test_sen = X_test.fillna("").apply(lambda row: " [SEP] ".join(row.values.astype(str)), axis = 1)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels = 3)
desc = model.cuda()

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
# Larger batch sizes tend to be better
batch_size = 32
# Use smaller learning rate to combat over-fitting
learning_rate = 1e-5
# Number of training epochs. 
epochs = 5
max_len = []

# For every sentence...
for sent in X_train_sen:
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    # Update the maximum sentence length.
    max_len.append(len(input_ids))

In [ ]:
max_len = int(np.percentile(np.array(max_len), 99))
max_len

271

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
train_input_ids = []
val_input_ids = []
test_input_ids = []
train_attention_masks = []
val_attention_masks = []
test_attention_masks = []

start = time.time()
print('Encoding all text in training...')
# For every sentence...
for sent in X_train_sen:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                )
    
    # Add the encoded sentence to the list.    
    train_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (differentiates padding from non-padding).
    train_attention_masks.append(encoded_dict['attention_mask'])

print(f'Time elapsed: {round(time.time() - start, 1)} seconds')
start = time.time()

print('Encoding all text in validation...')
for sent in X_val_sen:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt',
                )
    val_input_ids.append(encoded_dict['input_ids'])
    val_attention_masks.append(encoded_dict['attention_mask'])

print(f'Time elapsed: {round(time.time() - start, 1)} seconds')
start = time.time()

print('Encoding all text in test...')
for sent in X_test_sen:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = max_len,
                        truncation = True,
                        padding = 'max_length',
                        return_attention_mask = True,
                        return_tensors = 'pt',
                )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

print(f'Time elapsed: {round(time.time() - start, 1)} seconds')

# Convert the lists into tensors.
train_input_ids = torch.cat(train_input_ids, dim=0)
val_input_ids = torch.cat(val_input_ids, dim=0)
test_input_ids = torch.cat(test_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
val_attention_masks = torch.cat(val_attention_masks, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)
train_labels = torch.tensor(np.array(y_train))
train_labels = torch.reshape(train_labels, [train_labels.shape[0], 1])
val_labels = torch.tensor(np.array(y_val))
val_labels = torch.reshape(val_labels, [val_labels.shape[0], 1])
test_labels = torch.tensor(np.array(y_test))
test_labels = torch.reshape(test_labels, [test_labels.shape[0], 1])

print('DONE.')


Encoding all text in training...
Time elapsed: 86.1 seconds
Encoding all text in validation...
Time elapsed: 20.2 seconds
Encoding all text in test...
Time elapsed: 37.8 seconds
DONE.


In [ ]:
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

In [ ]:
# Create the DataLoaders for all datasets
# take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation and test the order doesn't matter, read them sequentially.
val_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate, 
                  eps = 1e-8 
                )
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 2352

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. 
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack training batch from dataloader
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # clear any previously calculated gradients before performing a
        model.zero_grad()        

        # Perform a forward pass 
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in val_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(val_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    torch.save(model, f"/content/drive/My Drive/Colab Notebooks/clinical_bert_{epoch_i}")
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...
  Batch   100  of  2,153.    Elapsed: 0:01:01.
  Batch   200  of  2,153.    Elapsed: 0:02:01.
  Batch   300  of  2,153.    Elapsed: 0:03:02.
  Batch   400  of  2,153.    Elapsed: 0:04:03.
  Batch   500  of  2,153.    Elapsed: 0:05:04.
  Batch   600  of  2,153.    Elapsed: 0:06:04.
  Batch   700  of  2,153.    Elapsed: 0:07:05.
  Batch   800  of  2,153.    Elapsed: 0:08:06.
  Batch   900  of  2,153.    Elapsed: 0:09:06.
  Batch 1,000  of  2,153.    Elapsed: 0:10:07.
  Batch 1,100  of  2,153.    Elapsed: 0:11:08.
  Batch 1,200  of  2,153.    Elapsed: 0:12:08.
  Batch 1,300  of  2,153.    Elapsed: 0:13:09.
  Batch 1,400  of  2,153.    Elapsed: 0:14:10.
  Batch 1,500  of  2,153.    Elapsed: 0:15:10.
  Batch 1,600  of  2,153.    Elapsed: 0:16:11.
  Batch 1,700  of  2,153.    Elapsed: 0:17:12.
  Batch 1,800  of  2,153.    Elapsed: 0:18:12.
  Batch 1,900  of  2,153.    Elapsed: 0:19:13.
  Batch 2,000  of  2,153.    Elapsed: 0:20:14.
  Batch 2,100  of

In [ ]:
pd.set_option('precision', 3)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.355,0.434,0.840,0:21:46,0:01:48
2,0.373,0.390,0.858,0:21:45,0:01:48
3,0.349,0.395,0.856,0:21:44,0:01:48
4,0.332,0.403,0.854,0:21:44,0:01:48
5,0.324,0.403,0.854,0:21:45,0:01:48


In [ ]:
torch.load("/content/drive/My Drive/Colab Notebooks/clinical_bert_1")

In [ ]:
prediction_sampler = SequentialSampler(test_dataset)
prediction_dataloader = DataLoader(test_dataset, sampler=prediction_sampler, batch_size=batch_size)

print('Predicting labels for {:,} records...'.format(len(test_dataset)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions.
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 21,521 records...
    DONE.


In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

print(classification_report(flat_true_labels, flat_predictions, digits = 3))


              precision    recall  f1-score   support

           0      0.889     0.957     0.922     17158
           1      0.671     0.473     0.555      3658
           2      0.590     0.384     0.466       705

    accuracy                          0.856     21521
   macro avg      0.717     0.605     0.647     21521
weighted avg      0.842     0.856     0.844     21521



Overall accuracy is about the same as the blended model, however there is a higher recall and higher f1 scores for those with a serious side effect or died.